# import 

In [9]:
import pandas as pd
import os
import numpy as np
import gc

In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings

warnings.filterwarnings('ignore')

In [35]:
import sys  # System-specific parameters and functions
import subprocess  # Spawn new processes, connect to their input/output/error pipes, and obtain their return codes
import os  # Operating system dependent functionality
import gc  # Garbage Collector interface
from pathlib import Path  # Object-oriented filesystem paths
from glob import glob  # Unix style pathname pattern expansion

import numpy as np  # Fundamental package for scientific computing with Python
import pandas as pd  # Powerful data structures for data manipulation and analysis
import polars as pl  # Fast DataFrame library implemented in Rust

from datetime import datetime  # Basic date and time types
import seaborn as sns  # Statistical data visualization
import matplotlib.pyplot as plt  # MATLAB-like plotting framework

import joblib  # Save and load Python objects

import warnings  # Warning control
warnings.filterwarnings('ignore')  # Ignore warnings

from sklearn.base import BaseEstimator, RegressorMixin  # Base classes for all estimators in scikit-learn
from sklearn.metrics import roc_auc_score  # ROC AUC score
import lightgbm as lgb  # LightGBM: Gradient boosting framework
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold  # Cross-validation strategies
from imblearn.over_sampling import SMOTE  # Oversampling technique for imbalanced datasets
from sklearn.preprocessing import OrdinalEncoder  # Encode categorical features as an integer array
from sklearn.impute import KNNImputer  # Imputation for completing missing values using k-Nearest Neighbors



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/MyDrive/SuperAI/Hackathon_l2/week5_KBTG_credit/parquet_files.zip /content/
!cp /content/drive/MyDrive/SuperAI/Hackathon_l2/week5_KBTG_credit/home-credit-credit-risk-modeling.zip /content/

In [6]:
!unzip /content/parquet_files.zip
!unzip /content/home-credit-credit-risk-modeling.zip

Archive:  /content/parquet_files.zip
  inflating: test/test_applprev_1_0.parquet  
  inflating: test/test_applprev_1_1.parquet  
  inflating: test/test_applprev_1_2.parquet  
  inflating: test/test_applprev_2.parquet  
  inflating: test/test_base.parquet  
  inflating: test/test_credit_bureau_a_1_0.parquet  
  inflating: test/test_credit_bureau_a_1_1.parquet  
  inflating: test/test_credit_bureau_a_1_2.parquet  
  inflating: test/test_credit_bureau_a_1_3.parquet  
  inflating: test/test_credit_bureau_a_1_4.parquet  
  inflating: test/test_credit_bureau_a_2_0.parquet  
  inflating: test/test_credit_bureau_a_2_1.parquet  
  inflating: test/test_credit_bureau_a_2_10.parquet  
  inflating: test/test_credit_bureau_a_2_11.parquet  
  inflating: test/test_credit_bureau_a_2_2.parquet  
  inflating: test/test_credit_bureau_a_2_3.parquet  
  inflating: test/test_credit_bureau_a_2_4.parquet  
  inflating: test/test_credit_bureau_a_2_5.parquet  
  inflating: test/test_credit_bureau_a_2_6.parquet  

# Function

In [7]:
import pandas as pd

def set_table_dtypes(df):
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
            df[col] = df[col].astype('Int64')
        elif col in ["date_decision"]:
            df[col] = pd.to_datetime(df[col])
        elif col[-1] in ("P", "A"):
            df[col] = df[col].astype(float)
        elif col[-1] in ("D",):
            df[col] = pd.to_datetime(df[col])
    return df

def convert_strings(df):
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            df[col] = df[col].astype(pd.CategoricalDtype(categories=new_categories, ordered=True))
    return df


In [8]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    return df


In [49]:

class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                continue
                df = df.with_columns(pl.col(col).cast(pl.str))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):

        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (col[-1] in ("M",)):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 300):
                    df = df.drop(col)

        return df




In [10]:

class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return  expr_max

    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return  expr_max

    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return  expr_max

    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return  expr_max

    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs



In [11]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    return df



In [12]:
def read_files(regex_path, depth=None):
    chunks = []

    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)

    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df



In [13]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols



In [14]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base



In [15]:
def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
        use.append(vx)
    return use



In [16]:
def group_columns_by_correlation(matrix, threshold=0.8):
    correlation_matrix = matrix.corr()
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]

    return groups



# Feature Engineer in Train

In [29]:
ROOT            = Path("/content")
ROOT2            = Path("/content")

TRAIN_DIR       = ROOT / "train"
TEST_DIR        = ROOT2 / "test_dataset" / "transformed"

set path train set to use

In [75]:

data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "/content/train/train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_0.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_0.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
    ]
}



Feature eng process

In [76]:
df_train = feature_eng(**data_store)
del data_store
gc.collect()


df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)


nums=df_train.select_dtypes(exclude='category').columns


from itertools import combinations, permutations

nans_df = df_train[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()



Feature use

In [77]:
uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            grps = group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
    else:
        uses=uses+v

# Subset the DataFrame to keep only the selected columns
df_train = df_train[uses]

main feature

In [ ]:
df_train=reduce_mem_usage(df_train)

# y = df_train["target"]
df_train= df_train.drop(columns=["case_id", "WEEK_NUM"])

In [81]:
df_train

,target,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_867L,clientscnt_1022L,...,max_childnum_185L,max_amount_416A,max_openingdate_313D,max_num_group1_9,max_contractenddate_991D,max_last180dayaveragebalance_704A,max_last180dayturnover_1134A,max_last30dayturnover_651A,max_openingdate_857D,max_num_group1_10
0,0,1,4,30000.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,4,19999.800781,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,5,78000.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,4,40000.000000,0.0,1.0,0.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,5,44000.000000,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,0,10,1,30000.000000,0.0,0.0,0.0,0.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526655,0,10,1,100000.000000,0.0,0.0,5.0,0.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526656,0,10,1,60000.000000,0.0,0.0,0.0,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526657,0,10,1,6000.000000,0.0,0.0,36.0,0.0,9.0,1.0,...,NaN,44916.644531,-1956.0,1.0,-861.0,NaN,NaN,NaN,-1956.0,1.0


In [82]:
train_col_list = df_train.columns.tolist()
for i in train_col_list:
  if i in ['isbidproduct_1095L']:
    print(df_train[i].dtype.name)

float16


In [64]:
df_train['isbidproduct_1095L']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1526654    1.0
1526655    1.0
1526656    1.0
1526657    1.0
1526658    1.0
Name: isbidproduct_1095L, Length: 1526659, dtype: float16

In [83]:
df_train.to_csv('train.csv', index=False)

# Feature eng in Test

In [84]:

data_store = {
    "df_base": read_file("/content/test.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_0.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_0.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
    ]
}

In [85]:
df_test = feature_eng(**data_store)
del data_store

df_test

case_id,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,…,max_contaddr_matchlist_1032L,max_contaddr_smempladdr_334L,max_empl_employedtotal_800L,max_empl_industry_691L,max_familystate_447L,max_gender_992L,max_housetype_905L,max_housingtype_772L,max_incometype_1044T,max_isreference_387L,max_maritalst_703L,max_personindex_1023L,max_persontype_1072L,max_persontype_792L,max_relationshiptoclient_415T,max_relationshiptoclient_642T,max_remitter_829L,max_role_1084L,max_role_993L,max_safeguarantyflag_411L,max_sex_738L,max_type_25L,max_num_group1_8,max_amount_416A,max_contractenddate_991D,max_openingdate_313D,max_num_group1_9,max_last180dayaveragebalance_704A,max_last180dayturnover_1134A,max_last30dayturnover_651A,max_openingdate_857D,max_num_group1_10,max_pmts_dpdvalue_108P,max_pmts_pmtsoverdue_635A,max_pmts_date_1107D,max_num_group1_11,max_num_group2
i64,i8,i8,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,…,bool,bool,str,str,str,str,str,str,str,bool,str,f64,f64,f64,str,str,bool,str,str,bool,str,str,i64,f64,i64,i64,i64,f64,f64,f64,i64,i64,f64,f64,i64,i64,i64
14256,5,6,null,null,-9435,null,564321.02,-26043,null,3.0,3.0,1.0,13.0,3.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",11.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.0,"""3439d993""",…,false,false,null,null,null,null,"""OWNED""",null,"""RETIRED_PENSIO…",null,null,0.0,1.0,1.0,null,null,null,"""CL""",null,true,"""M""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1348,11,5,null,null,null,null,null,-11452,null,2.0,2.0,1.0,2.0,2.0,"""a55475b1""","""a55475b1""","""a55475b1""",4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,"""a55475b1""",…,false,false,null,null,null,null,null,null,"""SALARIED_GOVT""",null,null,0.0,1.0,1.0,null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
13475,5,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,false,false,null,null,null,null,"""OWNED""",null,"""RETIRED_PENSIO…",null,null,0.0,1.0,1.0,null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1120,11,7,null,null,null,null,null,-9659,-9659,4.0,4.0,2.0,9.0,4.0,"""a55475b1""","""a55475b1""","""a55475b1""",14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,2.0,"""a7fcb6e5""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,1.4,3,2,9
11878,5,2,null,null,null,null,730076.77,-9334,null,4.0,5.0,2.0,11.0,4.0,"""2fc785b2""","""39a0853f""","""a55475b1""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.0,"""a7fcb6e5""",…,false,false,null,null,null,null,"""OWNED""",null,"""EMPLOYED""",null,null,0.0,1.0,1.0,null,null,null,"""CL""",null,true,null,"""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3077,11,6,null,null,null,null,null,-11241,null,2.0,2.0,0.0,4.0,1.0,"""a55475b1""","""717ddd49""","""a55475b1""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.0,"""3439d993""",…,f

In [86]:
gc.collect()
df_test = df_test.select([col for col in df_train.columns if col != "target"])
df_test, cat_cols = to_pandas(df_test)
df_test['isbidproduct_1095L'] = df_test['isbidproduct_1095L'].astype(float)
df_test = reduce_mem_usage(df_test)
gc.collect()

df_test

,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_867L,clientscnt_1022L,clientscnt_100L,...,max_childnum_185L,max_amount_416A,max_openingdate_313D,max_num_group1_9,max_contractenddate_991D,max_last180dayaveragebalance_704A,max_last180dayturnover_1134A,max_last30dayturnover_651A,max_openingdate_857D,max_num_group1_10
0,5,6,100000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,5,150000.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,6,19998.000000,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,7,50400.000000,0.0,1.0,0.0,0.0,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,31551.800781,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,11,5,11538.000000,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,4,5,140000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,11,4,74000.000000,0.0,0.0,0.0,0.0,11.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,4,5,10998.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
test_col_list = df_test.columns.tolist()
for i in test_col_list:
  if df_test[i].dtype.name in ['category']:
    print(i)

In [89]:
df_test.to_csv('test.csv')